In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import prepare_drafts as drafts


def create_lookup(df: pd.DataFrame, col: str, lookup_type: str) -> pd.DataFrame:
    """ Creates a section of the lookups data. """
    df = df.copy()
    
    keep_vars = ['draft_year', col]
    df = df_w_rank[keep_vars].drop_duplicates(subset=keep_vars)

    df.rename(columns={col: 'drafts_val'}, inplace=True)
    df['ranks_val'] = df['drafts_val']
    df['lookup_type'] = lookup_type

    df = df.loc[df['drafts_val'].notnull()]

    df = df[['lookup_type', 'draft_year', 'drafts_val', 'ranks_val']]

    return df


def export_lookups(df: pd.DataFrame, folder_path: str, file_name: str) -> None:
    df_player = create_lookup(df, 'full_name', 'player')
    df_team = create_lookup(df, 'team_name', 'team')
    df_pos = create_lookup(df, 'position', 'position')

    df = pd.concat([df_player, df_team, df_pos])

    df.to_csv(path.join(folder_path, file_name), index=False)


pd.set_option('display.max_rows', 300)

DRAFTS_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model/data'

df_drafts = drafts.read_raw_data(DRAFTS_FOLDER, [2021])
df_updated_types = drafts.update_dtypes(df_drafts)
df_complete_players = drafts.drafts_w_player_data(df_updated_types)
df_draft_attrs = drafts.add_draft_attrs(df_complete_players)
df_w_rank = drafts._add_rank_derived(df_draft_attrs)

In [18]:
##############################################################################################
############################# Create Custom Lookups ##########################################
##############################################################################################

OUTPUT_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model\
/data/2021'
export_lookups(df_w_rank, OUTPUT_FOLDER, 'lookups_2021.csv')


In [4]:
##############################################################################################
############################### Create Custom Ranks ##########################################
##############################################################################################

keep_vars = ['full_name', 'position', 'team_name', 'projection_adp'
            , 'rank_derived', 'draft_date']
rename_vars = {'full_name': 'player', 'position': 'pos', 'team_name': 'team'
            , 'projection_adp': 'adp', 'rank_derived': 'rank', 'draft_date': 'date'}

df = df_w_rank[keep_vars].copy()
df.rename(columns=rename_vars, inplace=True)

df['type'] = 'custom'

OUTPUT_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model\
/data/2021/player_ranks'
df.to_csv(path.join(OUTPUT_FOLDER, 'df_player_ranks_2021.csv'), index=False)

# df